<a href="https://colab.research.google.com/github/samuelj7/big-data-challenge/blob/main/Big_Data_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [ ]:
#Make connection to postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-23 07:29:57--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.20MB/s    in 0.2s    

2022-05-23 07:29:57 (5.20 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
# Load in .csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Digital_Music_Purchase_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10293599|R14LVB34Z2Z53I|B000YMOQZY|     262935067|Knockin' On Heave...|Digital_Music_Pur...|          5|            0|          0|   N|                Y|           favorites|Great  rendition....| 2015-08-31|
|         US|    6087195|R2M0R9CWPC82LT|B00ISAEC80|     452318038|            Flawless|Digital_Music_Pur

In [ ]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

1688884
1688606
1688606


In [ ]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
from pyspark.sql.functions import col

In [ ]:
from pyspark.sql.functions import to_date

In [ ]:
df_v2 = df.withColumn('review_date', to_date('review_date', 'yyyy-MM-dd'))
df_v2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10883775|R100C51VV2VHDO|B00TU1DE1M|     852458999|           Acid Rain|Digital_Music_Pur...|          5|            0|          0|   N|                Y|          Five Stars|Love it can't  wa...| 2015-03-26|
|         US|   19784609|R100JA85U1VLS2|B00136PPVO|      26005784|A Little Bit (Alb...|Digital_Music_Pur

In [ ]:
df_v2.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
df_v3 = df_v2.drop("marketplace", "product_category", "verified_purchase", "review_headline", "review_body")
df_v3.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   10883775|R100C51VV2VHDO|B00TU1DE1M|     852458999|           Acid Rain|          5|            0|          0|   N| 2015-03-26|
|   19784609|R100JA85U1VLS2|B00136PPVO|      26005784|A Little Bit (Alb...|          5|            0|          0|   N| 2012-10-11|
|   34449099|R10124X73KCB3Y|B0085LHUCC|     599118673|See You In September|          5|            0|          0|   N| 2013-12-23|
|   21317917|R10176O229WK9C|B000U8O4WM|     151883093|    Caribbean Breeze|          5|            1|          1|   N| 2014-07-01|
|   11981312|R1034NZ7GBLAGL|B004LY53SU|     884755125|           Yo No Fui|        

In [ ]:
df_v3.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- review_date: date (nullable = true)



In [ ]:
#Customer Table

df_v4 = df_v3.groupBy("customer_id").count()
customer=df_v4.withColumnRenamed("count", "customer_count")
customer.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   15293432|             3|
|   17022872|            10|
|   14206265|            78|
|   14202452|             1|
|   14563706|             1|
|   36528522|             7|
|   43783816|            18|
|   36169975|             1|
|   48335968|           134|
|   33144642|             5|
|   21880591|             1|
|   34196920|             3|
|   49352251|             1|
|   51343298|             1|
|    7418862|             1|
|   52129256|             7|
|   51803438|             4|
|   23691759|             1|
|   35033769|             7|
|   19108371|             4|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Review ID Table
review_id_table = df_v3.select(["review_id", "customer_id", "product_id", "product_parent", "review_date" ])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100C51VV2VHDO|   10883775|B00TU1DE1M|     852458999| 2015-03-26|
|R100JA85U1VLS2|   19784609|B00136PPVO|      26005784| 2012-10-11|
|R10124X73KCB3Y|   34449099|B0085LHUCC|     599118673| 2013-12-23|
|R10176O229WK9C|   21317917|B000U8O4WM|     151883093| 2014-07-01|
|R1034NZ7GBLAGL|   11981312|B004LY53SU|     884755125| 2015-08-06|
|R103XSHUPLDKLS|   12549169|B00GBCMW9S|     530484605| 2013-11-09|
| R104P6WTTR8IT|   16700850|B006YJMWRU|     169114284| 2012-10-07|
|R104Y1XZNVZV3N|   17768040|B000YGK9N8|     372899912| 2014-10-14|
|R1078H2CWGRBFB|   14681083|B0013HB6SE|     686465368| 2015-08-14|
|R107D1FGJPDHNM|   17267946|B001NTI8WY|     712339004| 2015-08-09|
|R1089QB3DHU7L7|   14240411|B00138ID7U|     750101767| 2013-12-30|
|R108HHSB8V0WB3|   43738799|B00ECJ7KAS|     514435493| 2013-08

In [ ]:
#Products Table
products = df_v3.select(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00TU1DE1M|           Acid Rain|
|B00136PPVO|A Little Bit (Alb...|
|B0085LHUCC|See You In September|
|B000U8O4WM|    Caribbean Breeze|
|B004LY53SU|           Yo No Fui|
|B00GBCMW9S|The Marshall Math...|
|B006YJMWRU|King Of Wishful T...|
|B000YGK9N8|You Can't Fix Stupid|
|B0013HB6SE|The Wabash Cannon...|
|B001NTI8WY|Weird Science (Al...|
|B00138ID7U|  How Great Thou Art|
|B00ECJ7KAS|O.M.O (Own My Own...|
|B0096RLJTY|     House Of Groove|
|B0031YEBGG|Santa Claus Is Co...|
|B00A8AFP66|Heart Of The Univ...|
|B004JENU50|        Lakota Tears|
|B0028EKUR4|       The Magic Key|
|B00906I49E|         Human Being|
|B00EZ2FBVC|Connecting With Y...|
|B006GBEJRW|          The Teaser|
+----------+--------------------+
only showing top 20 rows



In [ ]:
#Vine Table
vine = df_v3.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine" ])
vine.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100C51VV2VHDO|          5|            0|          0|   N|
|R100JA85U1VLS2|          5|            0|          0|   N|
|R10124X73KCB3Y|          5|            0|          0|   N|
|R10176O229WK9C|          5|            1|          1|   N|
|R1034NZ7GBLAGL|          5|            0|          0|   N|
|R103XSHUPLDKLS|          5|            3|          5|   N|
| R104P6WTTR8IT|          5|            0|          0|   N|
|R104Y1XZNVZV3N|          4|            0|          0|   N|
|R1078H2CWGRBFB|          5|            0|          0|   N|
|R107D1FGJPDHNM|          5|            0|          0|   N|
|R1089QB3DHU7L7|          2|            0|          0|   N|
|R108HHSB8V0WB3|          5|            0|          0|   N|
|R109AS4NXHR3JK|          5|            0|          0|   N|
|R10AQSQ9779IDI|          5|            

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://<insert endpoint>:5432/my_data_class_db"
config = {"user":"postgres",
          "password": "<postgres>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to table (Vine)

vine.write.jdbc(url=jdbc_url, table="vine", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Customer)


customer.write.jdbc(url=jdbc_url, table="customer", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Review ID)


review_id_table.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)

In [ ]:
# Write  DataFrame to table (Products)

products.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)